In [45]:
import numpy as np
import pandas as pd
import yfinance as yf
import concurrent.futures
import cvxpy as cp
import matplotlib.pyplot as plt
import logging

class OptionsTrader:
    def __init__(self, initial_cash=100000, desired_position_size=1, profit_target=0.2, stop_loss=0.1):
        self.initial_cash = initial_cash
        self.desired_position_size = desired_position_size
        self.profit_target = profit_target
        self.stop_loss = stop_loss

        self.portfolio = {'cash': initial_cash, 'positions': []}
        self.logger = logging.getLogger(__name__)
        self.asset_values = []
        self.cash_balances = []
        self.total_values = []
        self.selected_strategies = {}
        self.dates = []
        self.transactions = pd.DataFrame(columns=['date', 'type', 'price', 'strategy', 'num_shares', 'ticker'])
        
    def collect_information(self):
        return {
            'portfolio': self.portfolio,
            'asset_values': self.asset_values,
            'cash_balances': self.cash_balances,
            'total_values': self.total_values,
            'transactions': self.transactions,
            'dates': self.dates,
        }
    def fetch_options_data(self, ticker):
        options_data = {}
        exp_dates = yf.Ticker(ticker).options
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            future_to_exp_date = {executor.submit(self.fetch_single_option_chain, ticker, exp_date): exp_date for exp_date in exp_dates}
            for future in concurrent.futures.as_completed(future_to_exp_date):
                exp_date = future_to_exp_date[future]
                try:
                    exp_date, data = future.result()
                    if data is not None:
                        options_data[exp_date] = data
                except Exception as e:
                    self.logger.error(f"Error fetching options data for {ticker} on {exp_date}: {e}")
        return options_data

    def fetch_single_option_chain(self, ticker, exp_date):
        try:
            options_chain = yf.Ticker(ticker).option_chain(exp_date)
            calls = options_chain.calls[['strike', 'lastPrice']].rename(columns={'lastPrice': 'call_premium'})
            puts = options_chain.puts[['strike', 'lastPrice']].rename(columns={'lastPrice': 'put_premium'})
            merged = calls.set_index('strike').join(puts.set_index('strike'))
            return exp_date, merged
        except Exception as e:
            self.logger.error(f"Error fetching options data for {ticker} on {exp_date}: {e}")
            return exp_date, None

    def run_trader(self, ticker1, ticker2, start_date, end_date):
        self.ticker1 = ticker1
        self.ticker2 = ticker2
        self.start_date = start_date
        self.end_date = end_date

        try:
            historical_data1 = yf.download(ticker1, start=start_date, end=end_date)
            historical_data2 = yf.download(ticker2, start=start_date, end=end_date)
            if historical_data1.empty or historical_data2.empty:
                self.logger.error(f"No historical data found for {ticker1} or {ticker2}")
                return
        except Exception as e:
            self.logger.error(f"Error fetching historical data for {ticker1} or {ticker2}: {e}")
            return

        historical_close1 = historical_data1['Close']
        historical_close2 = historical_data2['Close']
        sigma1 = self.calculate_real_sigma(historical_close1)
        sigma2 = self.calculate_real_sigma(historical_close2)
        options_data1 = self.fetch_options_data(ticker1)
        options_data2 = self.fetch_options_data(ticker2)

        for current_date in pd.date_range(start=start_date, end=end_date, freq='B'):
            current_date_str = current_date.strftime('%Y-%m-%d')
            if current_date_str in historical_data1.index and current_date_str in historical_data2.index:
                current_price1 = historical_data1.loc[current_date_str, 'Close']
                current_price2 = historical_data2.loc[current_date_str, 'Close']

                volatility_factor = self.calculate_volatility_factor(historical_close1, historical_close2)

                for exp_date, data in options_data1.items():
                    try:
                        if data is None:
                            continue

                        T = (pd.to_datetime(exp_date) - pd.to_datetime(current_date)).days / 365
                        self.evaluate_and_store_best_strategy(data, ticker1, current_price1, T, sigma1, volatility_factor)

                    except Exception as e:
                        self.logger.error(f"Error evaluating strategies for {ticker1} on {exp_date}: {e}")
                        continue

                for exp_date, data in options_data2.items():
                    try:
                        if data is None:
                            continue

                        T = (pd.to_datetime(exp_date) - pd.to_datetime(current_date)).days / 365
                        self.evaluate_and_store_best_strategy(data, ticker2, current_price2, T, sigma2, volatility_factor)

                    except Exception as e:
                        self.logger.error(f"Error evaluating strategies for {ticker2} on {exp_date}: {e}")
                        continue

                self.allocate_capital(current_date_str, current_price1)
                self.allocate_capital(current_date_str, current_price2)

                self.check_sell_conditions(ticker1, current_price1, current_date_str)
                self.check_sell_conditions(ticker2, current_price2, current_date_str)

                self.update_portfolio_value(ticker1, current_price1, current_date_str)
                self.update_portfolio_value(ticker2, current_price2, current_date_str)

        self.flatten_positions(ticker1, current_price1, current_date_str)
        self.flatten_positions(ticker2, current_price2, current_date_str)
        self.update_portfolio_value(ticker1, current_price1, current_date_str)
        self.update_portfolio_value(ticker2, current_price2, current_date_str)

    def calculate_real_sigma(self, historical_prices):
        log_returns = np.log(historical_prices / historical_prices.shift(1)).dropna()
        sigma = np.std(log_returns) * np.sqrt(252)  # Annualize the volatility
        return sigma

    def calculate_volatility_factor(self, historical_prices1, historical_prices2):
        sigma1 = self.calculate_real_sigma(historical_prices1)
        sigma2 = self.calculate_real_sigma(historical_prices2)
        volatility_correlation = historical_prices1.pct_change().corr(historical_prices2.pct_change())
        return sigma1 * sigma2 * volatility_correlation

    def evaluate_and_store_best_strategy(self, options_data, ticker, S, T, sigma, volatility_factor):
        best_strategy = None
        best_return = -np.inf

        for strike in options_data.index:
            call_premium = options_data.loc[strike, 'call_premium']
            put_premium = options_data.loc[strike, 'put_premium']
            straddle_price = call_premium + put_premium
            expected_return = (self.simulate_straddle_return(S, strike, T, sigma) - straddle_price) * volatility_factor
            if expected_return > best_return:
                best_return = expected_return
                best_strategy = {
                    'strike': strike,
                    'expected_return': expected_return,
                    'straddle_price': straddle_price
                }

        if best_strategy:
            self.selected_strategies[ticker] = best_strategy
            self.logger.info(f"Best strategy for {ticker}: {best_strategy}")
        else:
            self.logger.warning(f"No suitable strategy found for {ticker}")

    def simulate_straddle_return(self, S, K, T, sigma, steps=100):
        return max(S - K, 0) + max(K - S, 0)

    def allocate_capital(self, current_date, current_price):
        if not self.selected_strategies:
            self.logger.warning("No strategies to allocate capital")
            return

        for ticker, strategy in self.selected_strategies.items():
            allocation = self.desired_position_size * strategy['straddle_price']

            if allocation < self.portfolio['cash']:
                allocation = self.desired_position_size * strategy['straddle_price']
                
                self.portfolio['cash'] -= allocation
                self.portfolio['positions'].append({
                    'ticker': ticker,
                    'allocation': allocation,
                    'strike': strategy['strike']
                })
                self.transactions = pd.concat([self.transactions, pd.DataFrame({'date': [pd.to_datetime(current_date)], 'type': ['buy'], 'price': [current_price], 'strategy': [strategy], 'ticker': [ticker]})], ignore_index=True)
    
                self.logger.info(f"Allocated ${allocation} to {ticker} strategy")
            else:
                self.logger.warning(f"Not enough cash to allocate to {ticker} strategy")

    def check_sell_conditions(self, ticker, current_price, current_date):
        for position in self.portfolio['positions']:
            if position['ticker'] == ticker:
                entry_price = position['allocation'] / position['strike']
                profit_loss = (current_price - entry_price) / entry_price

                if profit_loss >= self.profit_target or profit_loss <= -self.stop_loss:
                    self.portfolio['cash'] += current_price * position['strike']
                    self.portfolio['positions'].remove(position)
                    self.transactions = pd.concat([self.transactions, pd.DataFrame({'date': [pd.to_datetime(current_date)], 'type': ['sell'], 'price': [current_price], 'strategy': [position], 'ticker': [ticker]})], ignore_index=True)
                    self.logger.info(f"Sold {ticker} position at {current_price} with P/L {profit_loss}")

    def flatten_positions(self, ticker, current_price, current_date):
        for position in self.portfolio['positions']:
            if position['ticker'] == ticker:
                self.portfolio['cash'] += current_price * position['strike']
                self.transactions = pd.concat([self.transactions, pd.DataFrame({'date': [pd.to_datetime(current_date)], 'type': ['sell'], 'price': [current_price], 'strategy': [position], 'ticker': [ticker]})], ignore_index=True)
                self.logger.info(f"Flattened {ticker} position at {current_price}")
        self.portfolio['positions'] = []

    def update_portfolio_value(self, ticker, current_price, current_date):
        positions_value = sum(pos['allocation'] for pos in self.portfolio['positions'] if pos['ticker'] == ticker)

        total_value = self.portfolio['cash'] + positions_value
        self.asset_values.append(positions_value)
        self.cash_balances.append(self.portfolio['cash'])
        self.total_values.append(total_value)
        self.dates.append(current_date)

    def plot_portfolio_values(self):
        plt.figure(figsize=(14, 7))
        plt.plot(self.dates, self.total_values, label='Total Portfolio Value')
        plt.plot(self.dates, self.asset_values, label='Asset Value')
        plt.plot(self.dates, self.cash_balances, label='Cash Balance')
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title('Portfolio Values Over Time')
        plt.legend()
        plt.show()


In [46]:
trader = OptionsTrader()
trader.run_trader('AAPL', 'MSFT', '2021-01-01', '2021-12-31')


# Collect information about the trader
info = trader.collect_information()

# Print portfolio value and transactions
print("Portfolio Value over Time:")
print(info['total_values'])

print("\nTransactions:")
print(info['transactions'])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/jn/h8_6ldvx1wz2fwvbpv9k_nmh0000gn/T/ipykernel_82802/4014122100.py:183: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.transactions = pd.concat([self.transactions, pd.DataFrame({'date': [pd.to_datetime(current_date)], 'type': ['buy'], 'price': [current_price], 'strategy': [strategy], 'ticker': [ticker]})], ignore_index=True)


Portfolio Value over Time:
[307626.8836621094, 307836.4736621094, 647079.7530419922, 647289.3430419922, 977106.632126465, 977316.2221264649, 1316973.5029711917, 1317183.0929711917, 1659114.3789428712, 1659323.9689428713, 1996857.2629711914, 1997066.8529711915, 2331420.1360131837, 2331629.7260131836, 2668929.0112524414, 2669138.6012524413, 3001265.89857666, 3001475.48857666, 3332096.7708862303, 3332306.36088623, 3667889.6549145505, 3668099.2449145503, 4015682.5297875972, 4015892.119787597, 4367135.408322753, 4367344.998322753, 4721084.289055176, 4721293.879055176, 5081639.1664917, 5081848.7564917, 5445698.050886232, 5445907.640886232, 5809780.922097171, 5809990.512097171, 6178117.791110843, 6178327.381110843, 6535012.683195805, 6535222.273195805, 6902443.555505376, 6902653.145505376, 7270216.432209479, 7270426.022209479, 7641547.313674323, 7641756.903674323, 8013760.186716316, 8013969.776716316, 8385823.059758309, 8386032.649758309, 8758299.94342042, 8758509.53342042, 9131796.825251473,